In [ ]:
import cv2
import time
import tensorflow
import numpy as np
from tensorflow import keras
import matplotlib.pyplot as plt
from keras.utils import np_utils
from sklearn.preprocessing import MinMaxScaler
from keras.datasets import cifar10
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.utils import to_categorical , plot_model
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, precision_score, recall_score, f1_score
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten

In [ ]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

Display 10 images of X_train

In [ ]:
labels = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
fig, axes = plt.subplots(ncols=5, nrows=2)
index = 0
for i in range(2):
    for j in range(5):
        axes[i,j].set_title(labels[y_train[index][0]])
        axes[i,j].imshow(X_train[index])
        axes[i,j].get_xaxis().set_visible(False)
        axes[i,j].get_yaxis().set_visible(False)
        index += 1
plt.show()

Convert both X_train and X_test to grayscale

In [ ]:
X_train = np.array([cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) for image in X_train])
X_test = np.array([cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) for image in X_test])

Value normalization

In [ ]:
X_train  = X_train/255
X_test  = X_test/255

Defining input shape

In [ ]:
X_train = X_train.reshape(50000,1024)
X_test = X_test.reshape(10000,1024)

Label preprocessing

In [ ]:
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

Constructing MLP model

In [ ]:
def make_network():
  network = Sequential()
  network.add(Dense(256, activation = 'relu', input_shape = (1024, )))
  network.add(Dense(128, activation= 'relu'))
  network.add(Dense(64, activation= 'relu'))
  # network.add(Dense(32, activation= 'relu'))
  # network.add(Dense(16, activation= 'relu'))
  network.add(Dense(10, activation = 'softmax'))
  network.compile(loss = 'categorical_crossentropy', optimizer = keras.optimizers.RMSprop(learning_rate = 0.001), metrics= ['accuracy'])
  return network

In [ ]:
t1 = time.time()
network =make_network()
t2 = time.time()

Architecture of network

In [ ]:
plot_model(network, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

Model training

In [ ]:
history = network.fit(X_train, y_train,batch_size=32,epochs=50,validation_split = 0.1)

Model evaluation

In [ ]:
loss_plt = plt.figure(figsize = (10, 10))
plt.plot(history.history['accuracy'])
plt.plot(history.history["val_accuracy"])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(["Train accuracy", "Validation accuracy"])
plt.show()

Model evaluation

In [ ]:
loss_plt = plt.figure(figsize = (10, 10))
plt.plot(history.history['loss'])
plt.plot(history.history["val_loss"])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(["Train loss", "Validation loss"])
plt.show()

recall,precision,f1_score

In [ ]:
real_label = np.argmax(y_test, axis = 1)
predicted_label = np.argmax(network.predict(X_test), axis = 1)
recall = recall_score(real_label, predicted_label, average='micro')
precision = precision_score(real_label, predicted_label,average='macro')
f1 = f1_score(real_label, predicted_label,average='macro')
print("recall is :" , recall )
print("precision is :" , precision )
print("f1 is :",f1)


Loss and Accuracy in test data, Training time

In [ ]:
test_loss, test_acc = network.evaluate(X_test, y_test)
print("loss in test data is: ", test_loss)
print("accuracy in test data is : ", test_acc)
print("Training time is : ",t2 - t1)

Creating confusion matrix

In [ ]:
real_label = np.argmax(y_test, axis = 1)
predicted_label = np.argmax(network.predict(X_test), axis = 1)

cm = confusion_matrix(real_label, predicted_label)
normalized_cm = confusion_matrix(real_label, predicted_label, normalize = 'true')
fig, ax = plt.subplots(nrows = 1, ncols = 2)
fig.subplots_adjust(right = 3, top = 1.3)
ax[0].title.set_text("confusion Matrix - without Normalization")
ConfusionMatrixDisplay(cm).plot(cmap = plt.cm.Reds, values_format = 'd', xticks_rotation = 45, ax = ax[0])
ax[1].title.set_text("Confusion Matrix - Normalized")
ConfusionMatrixDisplay(normalized_cm).plot(cmap = plt.cm.Reds, values_format = ' .2f', xticks_rotation = 45, ax = ax[1])